<a href="https://colab.research.google.com/github/OdysseusPolymetis/digital_studies/blob/main/MUSE_for_latin_and_greek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**<center>Monolingual to multilingual mapping</center>**

Clonage de MUSE

In [ ]:
!git clone https://github.com/facebookresearch/MUSE.git

Imports

In [ ]:
import os
import cltk
import pickle
from gensim.models.fasttext import FastText
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np

Connexion au drive pour récupérer le corpus lemmatisé

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
with open('/content/drive/MyDrive/embeddings_save/lemmatized_greek_perseus.pkl', 'rb') as file:
    greek_corpus_processed = pickle.load(file)

In [ ]:
with open('/content/drive/MyDrive/embeddings_save/lemmatized_latin_perseus.pkl', 'rb') as file:
    latin_corpus_processed = pickle.load(file)

Vérification de la récupération

**Liste des auteurs avec leur clé :**
<br>Platon (tlg0059)
<br>Aristote (tlg0086)
<br>Héraclite (tlg0626)
<br>Démocrite (tlg1304)
<br>Pythagore (tlg0632)
<br>Antisthène (tlg0591)
<br>Parménide (tlg1562)
<br>Thales (tlg1705)
<br>Alcidamas (tlg0610)

<br>Cicéron (phi0474
<br>Sénèque (phi1017 0255 1014)
<br>Epictète (phi0557)
<br>Musonius Rufus (phi0628)
<br>Lucrèce (phi0550
<br>Marc Aurele (phi0562)
<br>Aulu Gelle (phi1254)
<br>Quintilien (phi1002)
<br>Boèce (phi0058)
<br>Varron (phi0684)
<br>Apulée (phi1212)

In [ ]:
# Endroit où stocker tous les auteurs disponibles
grc_target_authors = ["tlg0059","tlg0086","tlg1325", "tlg0626","tlg1304","tlg0632","tlg0591","tlg0593","tlg1562","tlg1705","tlg0014","tlg0610"]
lat_target_authors = ["phi0474", "phi1017","stoa0255","phi1014","tlg0557","phi0550","tlg0628","tlg0562","phi1254","phi1002","stoa0058","phi0684","phi1212"]

In [ ]:
#Première tranche latine
grc_target_authors = ["tlg0059","tlg0086","tlg1325", "tlg0626","tlg1304","tlg0632","tlg0591","tlg0593","tlg1562","tlg1705","tlg0014","tlg0610"]
lat_target_authors = ["phi0474", "phi1017","stoa0255","phi1014","tlg0557","phi0684"]

In [ ]:
#Seconde tranche latine
grc_target_authors = ["tlg0059","tlg0086","tlg1325", "tlg0626","tlg1304","tlg0632","tlg0591","tlg0593","tlg1562","tlg1705","tlg0014","tlg0610"]
lat_target_authors = ["phi0550", "tlg0628", "tlg0562", "phi1254", "phi1002", "stoa0058", "phi1212"]

In [ ]:
auteurs = [
    ("Thalès", -624, -546),
    ("Pythagore", -570, -495),
    ("Héraclite", -535, -475),
    ("Parménide", -515, -450),
    ("Alcidamas", -450, -400),  # Les dates pour Alcidamas sont approximatives
    ("Antisthène", -445, -365),
    ("Platon", -427, -347),
    ("Démocrite", -460, -370),
    ("Aristote", -384, -322),
    ("Cicéron", -106, -43),
    ("Lucrèce", -99, -55),
    ("Varron", -116, -27),
    ("Sénèque", -4, 65),
    ("Épictète", 50, 135),
    ("Musonius Rufus", 20, 101),
    ("Quintilien", 35, 100),
    ("Marc Aurèle", 121, 180),
    ("Apulée", 124, 170),
    ("Aulu Gelle", 125, 180),  # Les dates pour Aulu Gelle sont approximatives
    ("Boèce", 477, 524)
]

In [ ]:
auteurs = auteurs[::-1]

# Tracer le diagramme de Gantt
fig, ax = plt.subplots(figsize=(10, 12))
y_labels = []
y_ticks = []
for i, (auteur, naissance, mort) in enumerate(auteurs):
    ax.broken_barh([(naissance, mort-naissance)], (i-0.4, 0.8), facecolors=('orange'), edgecolor=('black'))
    y_labels.append(auteur)
    y_ticks.append(i)

ax.set_xlabel("Années")
ax.set_yticks(y_ticks)
ax.set_yticklabels(y_labels)
ax.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.title('Durée de vie des auteurs antiques')
plt.tight_layout()
plt.show()

Tri des textes sur la base des auteurs demandés

In [ ]:
lat_sentences = []
grc_sentences = []

for cle_auteur in lat_target_authors:
    for cle, valeur in latin_corpus_processed.items():
        if cle_auteur in cle:
            lat_sentences.extend(valeur)

for cle_auteur in grc_target_authors:
    for cle, valeur in greek_corpus_processed.items():
        if cle_auteur in cle:
            grc_sentences.extend(valeur)

Création des vecteurs, avec des dimensions équivalentes

In [ ]:
model_lat = FastText(vector_size=300, window=50, min_count=2)
model_grc = FastText(vector_size=300, window=50, min_count=2)
model_lat.build_vocab(lat_sentences)
model_lat.train(lat_sentences, total_examples=model_lat.corpus_count, epochs=10)
model_grc.build_vocab(grc_sentences)
model_grc.train(grc_sentences, total_examples=model_grc.corpus_count, epochs=10)

Vérification qu'il n'y a pas de clé vide

In [ ]:
similar_words = model_lat.wv.most_similar('')

Adaptation des vecteurs FastText pour MUSE

In [ ]:
def export_fasttext_vectors_to_text(model, output_path):
    # Ouvrir le fichier de sortie
    with open(output_path, 'w', encoding='utf-8') as f:
        # Écrire le nombre de mots et la taille des embeddings (dimension)
        f.write(f"{len(model.wv.index_to_key)} {model.vector_size}\n")

        # Pour chaque mot dans le vocabulaire du modèle
        for word in model.wv.index_to_key:
            # Remplacer les espaces dans le mot par des underscores
            sanitized_word = word.replace(' ', '_').lower()

            # Écrire le mot
            f.write(sanitized_word + ' ')

            # Écrire le vecteur associé à ce mot
            vector_str = ' '.join(map(str, model.wv[word]))
            f.write(vector_str + '\n')


In [ ]:
output_lat_path = '/content/embeddings_lat3.vec'
output_grc_path = '/content/embeddings_grc.vec'
export_fasttext_vectors_to_text(model_lat, output_lat_path)
export_fasttext_vectors_to_text(model_grc, output_grc_path)

Reprise des paires de mots : les données étant sous droit ou en passe d'être normalisées, elles ne sont pas accessibles ici. Pour paramétrer le dictionnaire, il vous faut des paires de mots sous cette forme:
<br><br>
`motSource \t motCible`

Préparation du dictionnaire pour MUSE (il ne doit pas y avoir de mots inconnus dans le dictionnaire)

In [ ]:
# Charger les embeddings
src_words = set()
with open("/content/embeddings_grc.vec", "r", encoding="utf-8") as f:
    next(f)  # skip the header line
    for line in f:
        word = line.split()[0]
        src_words.add(word)

tgt_words = set()
with open("/content/embeddings_lat3.vec", "r", encoding="utf-8") as f:
    next(f)  # skip the header line
    for line in f:
        word = line.split()[0]
        tgt_words.add(word)

# Filtrer le dictionnaire de supervision
with open("/content/all_pairs.tsv", "r", encoding="utf-8") as f, open("/content/filtered_pairs.txt", "w", encoding="utf-8") as fout:
    for line in f:
        values = line.strip().split("\t")  # Split by tab
        if len(values) < 2:  # Ensure there are at least 2 values
            print(f"Skipping malformed line: {line.strip()}")
            continue

        # Retrieve and clean the words
        src_word = values[0].replace("(grc)", "").strip()
        tgt_word = values[1].replace("(lat)", "").strip()

        if src_word in src_words and tgt_word in tgt_words:
            fout.write(f"{src_word.lower()}\t{tgt_word.lower()}\n")


Vérification du dictionnaire

In [ ]:
def load_embedding_words(filepath):
    """Chargez tous les mots d'un fichier d'embeddings."""
    with open(filepath, 'r', encoding='utf-8') as f:
        # Ignorez la première ligne (nombre de mots et dimensions)
        f.readline()
        # Récupérez tous les mots du fichier
        words = {line.split()[0] for line in f}
    return words

def check_words_in_embeddings(embedding_path, pair_path, column=0):
    """Vérifiez si les mots du fichier de paires sont présents dans les embeddings."""
    embedding_words = load_embedding_words(embedding_path)
    missing_words = []

    with open(pair_path, 'r', encoding='utf-8') as f:
        for line in f:
            word = line.strip().split()[column]  # Prenez le mot en fonction de la colonne
            if word not in embedding_words:
                missing_words.append(word)

    return missing_words

src_embedding_path = '/content/embeddings_grc.vec'
tgt_embedding_path = '/content/embeddings_lat3.vec'
pair_path = '/content/filtered_pairs.txt'

missing_src_words = check_words_in_embeddings(src_embedding_path, pair_path, column=0)
missing_tgt_words = check_words_in_embeddings(tgt_embedding_path, pair_path,column=1)

print("Mots manquants dans les embeddings source:", missing_src_words)
print("Mots manquants dans les embeddings cibles:", missing_tgt_words)

Vérification des fichiers de vecteurs

In [ ]:
def verify_vec_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        # lire la première ligne
        first_line = f.readline().strip().split()
        if len(first_line) != 2:
            return False, "La première ligne est mal formatée."

        num_words, dim = map(int, first_line)

        # vérifier chaque ligne
        for idx, line in enumerate(f, 1):
            tokens = line.strip().split()
            if len(tokens) != dim + 1:
                return False, f"Ligne {idx + 1} mal formatée. Attendu {dim + 1} tokens, obtenu {len(tokens)}"

    return True, "Fichier .vec valide."

# Testez vos fichiers
src_verification = verify_vec_file('/content/embeddings_grc.vec')
tgt_verification = verify_vec_file('/content/embeddings_lat3.vec')

print("Vérification des embeddings source:", src_verification)
print("Vérification des embeddings cibles:", tgt_verification)

## Lancement de MUSE

In [ ]:
!python /content/MUSE/supervised.py --src_emb /content/embeddings_grc.vec --tgt_emb /content/embeddings_lat3.vec --dico_train /content/filtered_pairs.txt --dico_eval /content/filtered_pairs.txt

In [ ]:
from gensim.models import KeyedVectors

# Chargement des embeddings
grc_vectors = KeyedVectors.load_word2vec_format('/content/MUSE/dumped/debug/1bq81qr2w1/vectors-en.txt')
lat_vectors = KeyedVectors.load_word2vec_format('/content/MUSE/dumped/debug/1bq81qr2w1/vectors-es.txt')

def get_most_similar(word, src_vectors, trg_vectors, topn=10):
    if word not in src_vectors:
        return []

    grc_vector = src_vectors[word]

    # Calculer la similarité avec chaque mot en latin
    similarities = {}
    for lat_word in trg_vectors.index_to_key:
        lat_vector = trg_vectors[lat_word]
        similarity = src_vectors.cosine_similarities(grc_vector, [lat_vector])[0]
        similarities[lat_word] = similarity

    sorted_words = sorted(similarities.keys(), key=lambda x: similarities[x], reverse=True)

    return sorted_words[:topn]

In [ ]:
print(get_most_similar("δόξα", grc_vectors, lat_vectors))